In [40]:
from datasets import load_dataset, Audio
import pandas as pd
import warnings
warnings.filterwarnings('always') 
import random
import requests
import gzip
from collections import Counter
import numpy as np
import io
import numpy.core.multiarray as multi
from sklearn.metrics import classification_report

# Standard scientific Python imports
import matplotlib.pyplot as plt

# Import datasets, classifiers and performance metrics
from sklearn import datasets, metrics, svm
from sklearn.model_selection import train_test_split
import soundfile as sf
import gzip
import torch
import matplotlib.pyplot as plt
from IPython.display import Audio as iAudio
from npc_gzip.compressors.gzip_compressor import GZipCompressor
from npc_gzip.knn_classifier import KnnClassifier


In [41]:
# Reference:
# Input: The Audio feature accepts as input:
#     A str: Absolute path to the audio file (i.e. random access is allowed).
#     A dict with the keys:
#         path: String with relative path of the audio file to the archive file.
#         bytes: Bytes content of the audio file.

# Convert the NumPy arrays to audio bytes in WAV format
def numpy_to_bytes(audio_array, sampling_rate=16000):
    with io.BytesIO() as bytes_io:
        sf.write(bytes_io, audio_array, samplerate=sampling_rate, format='WAV')
        return bytes_io.getvalue()

In [42]:
def get_data(dataset) -> tuple:
    """
    Pulls the Project CETI sperm whale vocalizations
    training data and the second being the test
    data. Each tuple contains the audio and label
    respectively as numpy arrays.

    """

    train_iter = dataset['train']
    test_iter = dataset['test']

    train_audio =  [numpy_to_bytes(audio_array['array']) for audio_array in train_iter['audio']]
    train_labels = train_iter['coda_type']
    
    test_audio  = [numpy_to_bytes(audio_array['array']) for audio_array in test_iter['audio']]
    test_labels = test_iter['coda_type']
    
    train_audio = np.array(train_audio)
    train_labels = np.array(train_labels)

    test_audio = np.array(test_audio)
    test_labels = np.array(test_labels)

    train = (train_audio, train_labels)
    test = (test_audio, test_labels)

    return (train, test)

In [43]:
def fit_model(train_audio: np.ndarray, train_labels: np.ndarray, distance_metric: str = "ncd") -> KnnClassifier:
    """
    Fits a Knn-GZip compressor on the train
    data and returns it.

    Arguments:
        train_text (np.ndarray): Training dataset as a numpy array.
        train_labels (np.ndarray): Training labels as a numpy array.

    Returns:
        KnnClassifier: Trained Knn-Compressor model ready to make predictions.
    """
    compressor: BaseCompressor = GZipCompressor()
    model: KnnClassifier = KnnClassifier(
        compressor=compressor,
        training_inputs=train_audio,
        training_labels=train_labels,
        distance_metric=distance_metric,
    )

    return model

In [44]:
dataset = load_dataset("autumnjohnson/ceti_audio")#.cast_column("audio", Audio(decode=False))

In [45]:
((train_audio, train_labels), (test_audio, test_labels)) = get_data(dataset)

In [ ]:
model = fit_model(train_audio, train_labels)

In [ ]:
random_indicies = np.random.choice(test_audio.shape[0], len(test_audio), replace=False)

In [ ]:
sample_test_text = test_audio[random_indicies]

In [ ]:
sample_test_labels = test_labels[random_indicies]

In [ ]:
 # Here we use the `sampling_percentage` to save time
 # at the expense of worse predictions. This
 # `sampling_percentage` selects a random % of training
 # data to compare `sample_test_text` against rather
 # than comparing it against the entire training dataset.
top_k = 1
(distances, labels, similar_samples) = model.predict(sample_test_text, top_k, sampling_percentage=0.01)

In [ ]:
flattened_labels = labels.flatten()

In [ ]:
p = Counter(flattened_labels)  

In [ ]:
l = Counter(sample_test_labels)

In [ ]:
x =(l['1+1+3']+p['1+1+3'])
l

In [ ]:
print(classification_report(sample_test_labels, flattened_labels.reshape(-1)))

In [ ]:
dataset['train']['audio'][0]['array'].decode("utf-8")

In [ ]:
def xfunc(c):
    bincount = np.bincount(c)
    argmax = np.argmax(bincount, axis=0)
    
    return argmax

In [ ]:
disp = metrics.ConfusionMatrixDisplay.from_predictions(sample_test_labels, flattened_labels.reshape(-1))
disp.figure_.suptitle("Confusion Matrix")
print(f"Confusion matrix:\n{disp.confusion_matrix}")

plt.show()

In [ ]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(sample_test_labels, flattened_labels.reshape(-1))